## CSRD Data Points 
- Processing CSRD datapoints into a list of dictionaries 

In [1]:
%pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 

In [9]:
df_sheets = pd.read_excel('data/esrs_data.xlsx', sheet_name=None)

In [16]:
sheet = df_sheets['ESRS 2']
sheet = sheet.drop(sheet.index[0], axis=0)
sheet.head()

,"INSTRUCTIONS\n1)The ""shall disclose"" DPs reported in ESRS 2 are to be disclosed irrespective of the outcome of its materiality assessment [ESRS 1 par. 29].\n2) Undertakings or groups not exceeding on their balance sheet dates the average number of 750 employees during the financial year (on a consolidated basis where applicable) may omit the information required by ESRS E4, ESRS S1, ESRS S2, ESRS S3 or ESRS S4 in accordance with Appendix C of ESRS 1. In this case, the undertaking shall nevertheless disclose DPs reported under par. 17-BP2 of ESRS 2 if one or more of these topics have been assessed to be material [see Column K].\n3) Column L identifies DPs related to SBM-1 and SBM-3 subject to phased in according to Appendix C of ESRS 1.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
1,BP-1_01,ESRS 2,BP-1,5 a,NaN,Basis for preparation of sustainability statement,semi-narrative,NaN,NaN,NaN,NaN,NaN
2,BP-1_02,ESRS 2,BP-1,5 b i,NaN,Scope of consolidation of consolidated sustain...,narrative,Conditional,NaN,NaN,NaN,NaN
3,BP-1_03,ESRS 2,BP-1,5 b ii,NaN,Indication of subsidiary undertakings included...,narrative,Conditional,NaN,NaN,NaN,NaN
4,BP-1_04,ESRS 2,BP-1,5 c,AR 1,Disclosure of extent to which sustainability s...,narrative,NaN,NaN,NaN,NaN,NaN
5,BP-1_05,ESRS 2,BP-1,5 d,NaN,Option to omit specific piece of information c...,semi-narrative,NaN,NaN,NaN,NaN,NaN
